# 8.0 — Diebold–Mariano : LSTM vs HAR vs EWM

**But :** comparer les prévisions LSTM et HAR-RV à l’EWM via le test Diebold–Mariano (MSE).


In [1]:
from pathlib import Path
import numpy as np, pandas as pd
from scipy.stats import norm

LSTM_DIR = Path("data/models/lstm")
HAR_DIR  = Path("data/models/har_rv")


### 8.1 Fonction "dm_mse"

**Explication :**
1) On calcule la différence des erreurs quadratiques entre deux modèles ("p1" et "p2") par rapport à la série réelle "y".  
2) On normalise cette différence pour obtenir la statistique de test de Diebold–Mariano (DM).  
3) On calcule la p-value associée avec une loi normale centrée réduite.  

**Pourquoi nous avons fait cela :**
1) Le test de Diebold–Mariano permet de comparer objectivement deux modèles de prévision sur leurs erreurs (ici via la MSE).  
2) Cela nous permet de savoir si un modèle apporte une amélioration significative par rapport à un autre.  


In [2]:
def dm_mse(y, p1, p2):
    d = (y - p1)**2 - (y - p2)**2
    dm = d.mean() / (d.std(ddof=1)/np.sqrt(len(d)))
    p  = 2*(1 - norm.cdf(abs(dm)))
    return float(dm), float(p)


### 8.2 Fonction "run"

**Explication :**
1) On charge les résultats du modèle LSTM et HAR-RV depuis leurs fichiers CSV.  
2) On fusionne les deux DataFrames sur la colonne "date" pour aligner les prédictions.  
3) On extrait les vraies valeurs (y_true), les prévisions EWM, LSTM et HAR.  
4) On applique le test de Diebold–Mariano pour comparer :  
   - HAR vs EWM  
   - LSTM vs EWM  
   - LSTM vs HAR  

**Pourquoi nous avons fait cela :**
1) Cela permet de tester statistiquement si LSTM ou HAR sont significativement meilleurs que le benchmark EWM.  
2) La comparaison LSTM vs HAR permet d’évaluer si l’apprentissage profond apporte un gain par rapport à un modèle économétrique classique.  


In [ ]:
def run(target_key, lstm_file, har_file):
    lstm = pd.read_csv(LSTM_DIR / f"{lstm_file}_test.csv", parse_dates=["date"])
    har  = pd.read_csv(HAR_DIR  / f"{har_file}.csv", parse_dates=["date"])
    df = lstm.merge(har[["date","y_pred_har"]], on="date", how="inner")
    y, ewm, lstm_pred, har_pred = df["y_true"].values, df["y_ewm"].values, df["y_lstm"].values, df["y_pred_har"].values
    print(f"\n== {target_key} ==")
    for a_name, a in [("HAR",har_pred), ("LSTM",lstm_pred)]:
        dm, p = dm_mse(y, a, ewm)
        print(f"DM (MSE): {a_name} vs EWM → stat={dm:.3f}, p={p:.4g}")
    dm_lh, p_lh = dm_mse(y, lstm_pred, har_pred)
    print(f"DM (MSE): LSTM vs HAR → stat={dm_lh:.3f}, p={p_lh:.4g}")




### 8.2 Fonction "run"

**Résultat pour sigma 1:**
- HAR est significativement meilleur que EWM.
- Aucune différence significative, LSTM n’apporte pas de gain clair sur EWM.
- Pas de différence significative entre LSTM et HAR → mais HAR est celui qui bat clairement EWM.
    - Conclusion : HAR domine EWM, LSTM ne fait pas mieux.

**Résultat pour vol5:**
- Aucune différence significative, HAR = EWM.
- LSTM est significativement meilleur que EWM.
- LSTM est significativement meilleur que HAR.
    - Conclusion : LSTM est supérieur à la fois à HAR et EWM pour vol5f.

In [ ]:
if __name__ == "__main__":
    run("sigma1", "lstm_target_sigma1_enriched_log", "har_sigma1_test")
    run("vol5f", "lstm_target_vol5f_enriched_log",   "har_vol5f_test")



== sigma1 ==
DM (MSE): HAR vs EWM → stat=-3.533, p=0.0004116
DM (MSE): LSTM vs EWM → stat=-1.009, p=0.3128
DM (MSE): LSTM vs HAR → stat=1.336, p=0.1815

== vol5f ==
DM (MSE): HAR vs EWM → stat=-0.123, p=0.9018
DM (MSE): LSTM vs EWM → stat=-4.248, p=2.153e-05
DM (MSE): LSTM vs HAR → stat=-4.289, p=1.796e-05
